### Functions for image processing

In [ ]:
def read_img(file_name):
    # Read Image and Gray
    img = cv2.imread(file_name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    return img, gray

def get_imgInfo(img):
    # Get image's height and width
    height, width = img.shape[:2]
    
    return height, width

def threshold(gray, height, width , percent):
    index = int(height*width*percent)
    thresh_value = np.sort(gray.ravel())[index]
    _, thresh = cv2.threshold(gray, thresh_value,255,cv2.THRESH_BINARY)
    
    return thresh

def median_filter(img, filter_zise=3):
    median = cv2.medianBlur(img, filter_zise)
    
    return median

def opening(img, size=3):
    kernel = np.ones((size,size),np.uint8)
    opened = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    
    return opened

def closing(img, size=3):
    kernel = np.ones((size,size),np.uint8)
    closed = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    return closed

def detect_circles(img):
    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,80,
                            param1=1,param2=20,minRadius=0,maxRadius=0)
    return circles

def draw_circles(img, circles):
    for i in circles[0,:]:
        # draw the outer circle
        cv2.circle(img,(i[0],i[1]),i[2],(0,255,0),1)
        # draw the center of the circle
        cv2.circle(img,(i[0],i[1]),2,(0,0,255),2)
        
        print("center: ({}, {})".format(i[0],i[1]))
    return img

def edge_detect(img, minVal, maxVal):
    edge = cv2.Canny(img, minVal, maxVal)
    
    return edge

def draw_lines(img, lines):
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 1)
        
def threshold_moment_preserving(img, gray):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)    
    H, S, B = cv2.split(img) # spliting the image into HSB


    #Instead of skimage.exposure.histogram (B, nbins=256)   
    grey_value = np.arange(256)
    B_freq = cv2.calcHist([S], [0],None,[256],[0,255])
    B_freq = B_freq.reshape((256,))
    B_freq = np.int64(B_freq)
    B_histo = (B_freq, grey_value)

    pix_sum = B.shape[0]*B.shape[1] # calculating the sum of the pixels

    #from the paper, calculating the 3 first odrers

    pj = B_histo[0] / pix_sum
    pj_z1 = np.power(B_histo[1], 1) * pj
    pj_z2 = np.power(B_histo[1], 2) * pj
    pj_z3 = np.power(B_histo[1], 3) * pj

    m0 = np.sum(pj)
    m1 = np.sum(pj_z1)
    m2 = np.sum(pj_z2)
    m3 = np.sum(pj_z3)

    cd = (m0*m2) - (m1*m1)
    c0 = ((-m2*m2) - (-m3*m1))/cd
    c1 = ((m0*-m3) - (m1*-m2))/cd


    z0 = 0.5 *(-c1 - (np.power(np.power(c1, 2) - 4*c0, 1/2)))
    z1 = 0.5 *(-c1 + (np.power(np.power(c1, 2) - 4*c0, 1/2)))

    pd = z1 - z0
    p0 = (z1 - m1) / pd # p0 should be the percentage of the pixels to which the threshold t should be done

    # using cumulative histogram and comparing it to a target value by calculating the difference. When the difference is the lowest, the index indicates the value of the threshold t
    cum_pix = np.cumsum(B_freq) 
    target_value = p0 * pix_sum

    diff = [(i - target_value) for i in cum_pix]

    def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        print(idx)
        return array[idx]

    t = diff.index(find_nearest(diff, 0))
    _, thresh_moment = cv2.threshold(gray, t,255,cv2.THRESH_BINARY)
    return thresh_moment

## Funtions for counting results

In [ ]:
def get_centroid(img):
    height, width = img.shape[:2]
    pointNumber = 0
    cx = 0
    cy = 0
    for i in range(height):
        for j in range(width):
            if img[i,j] == 255:
                pointNumber += 1
                cx += j
                cy += i
        
    return cx/pointNumber, cy/pointNumber

def get_max_length(predictions):
    maxLength = 0
    size = len(predictions) 
    for i in range(0,size-1):
        for j in range(i+1, size):
            x_diff, y_diff = (predictions[i][0] - predictions[j][0]), (predictions[i][1] - predictions[j][1])
            length = math.sqrt(x_diff**2 + y_diff**2)
            if (length > maxLength):
                maxLength = length

    return maxLength


def get_max_length_axil(predictions):
    x_max = 0
    y_max = 0
    size = len(predictions) 
    for i in range(0, size-1):
        for j in range(i+1, size):
            x_diff, y_diff = (predictions[i][0] - predictions[j][0]), (predictions[i][1] - predictions[j][1])
            if (x_max < x_diff):
                x_max = x_diff
            if (y_max < y_diff):
                y_max = y_diff
        
    return x_max, y_max

    
def show_resoult(predictions):
    mean = np.mean(predictions, axis=0)
    var  = np.var(predictions,  axis=0)
    x_max, y_max = get_max_length_axil(predictions)
    print("-----------------------------------------------------")
    print("Statistics: ")
    print("Mean Centroid: ({:.5f}, {:.5f})".format(mean[0], mean[1]))
    print("Variance     : ({:.5f}, {:.5f})".format(var[0], var[1]))
    print("Max Length   : {:.5f}, {:.5f} (pixels)".format(x_max, y_max))
    

In [ ]:
def max_entropy(data):
    """
    Implements Kapur-Sahoo-Wong (Maximum Entropy) thresholding method
    Kapur J.N., Sahoo P.K., and Wong A.K.C. (1985) "A New Method for Gray-Level Picture Thresholding Using the Entropy
    of the Histogram", Graphical Models and Image Processing, 29(3): 273-285
    M. Emre Celebi
    06.15.2007
    Ported to ImageJ plugin by G.Landini from E Celebi's fourier_0.8 routines
    2016-04-28: Adapted for Python 2.7 by Robert Metchev from Java source of MaxEntropy() in the Autothresholder plugin
    http://rsb.info.nih.gov/ij/plugins/download/AutoThresholder.java
    :param data: Sequence representing the histogram of the image
    :return threshold: Resulting maximum entropy threshold
    """

    # calculate CDF (cumulative density function)
    cdf = data.astype(np.float).cumsum()

    # find histogram's nonzero area
    valid_idx = np.nonzero(data)[0]
    first_bin = valid_idx[0]
    last_bin = valid_idx[-1]

    # initialize search for maximum
    max_ent, threshold = 0, 0

    for it in range(first_bin, last_bin + 1):
        # Background (dark)
        hist_range = data[:it + 1]
        hist_range = hist_range[hist_range != 0] / cdf[it]  # normalize within selected range & remove all 0 elements
        tot_ent = -np.sum(hist_range * np.log(hist_range))  # background entropy

        # Foreground/Object (bright)
        hist_range = data[it + 1:]
        # normalize within selected range & remove all 0 elements
        hist_range = hist_range[hist_range != 0] / (cdf[last_bin] - cdf[it])
        tot_ent -= np.sum(hist_range * np.log(hist_range))  # accumulate object entropy

        # find max
        if tot_ent > max_ent:
            max_ent, threshold = tot_ent, it

    return threshold

In [ ]:
def threshold_moment_preserving(img, gray):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)    
    H, S, B = cv2.split(img) # spliting the image into HSB


    #Instead of skimage.exposure.histogram (B, nbins=256)   
    grey_value = np.arange(256)
    B_freq = cv2.calcHist([S], [0],None,[256],[0,255])
    B_freq = B_freq.reshape((256,))
    B_freq = np.int64(B_freq)
    B_histo = (B_freq, grey_value)

    pix_sum = B.shape[0]*B.shape[1] # calculating the sum of the pixels

    #from the paper, calculating the 3 first odrers

    pj = B_histo[0] / pix_sum
    pj_z1 = np.power(B_histo[1], 1) * pj
    pj_z2 = np.power(B_histo[1], 2) * pj
    pj_z3 = np.power(B_histo[1], 3) * pj

    m0 = np.sum(pj)
    m1 = np.sum(pj_z1)
    m2 = np.sum(pj_z2)
    m3 = np.sum(pj_z3)

    cd = (m0*m2) - (m1*m1)
    c0 = ((-m2*m2) - (-m3*m1))/cd
    c1 = ((m0*-m3) - (m1*-m2))/cd


    z0 = 0.5 *(-c1 - (np.power(np.power(c1, 2) - 4*c0, 1/2)))
    z1 = 0.5 *(-c1 + (np.power(np.power(c1, 2) - 4*c0, 1/2)))

    pd = z1 - z0
    p0 = (z1 - m1) / pd # p0 should be the percentage of the pixels to which the threshold t should be done

    # using cumulative histogram and comparing it to a target value by calculating the difference. When the difference is the lowest, the index indicates the value of the threshold t
    cum_pix = np.cumsum(B_freq) 
    target_value = p0 * pix_sum

    diff = [(i - target_value) for i in cum_pix]

    def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        print(idx)
        return array[idx]

    t = diff.index(find_nearest(diff, 0))
    _, thresh_moment = cv2.threshold(gray, t,255,cv2.THRESH_BINARY)
    return thresh_moment

In [1]:
def removeBorder(img, height, width):
    minimum = int(0.001*height*width)
    hist, _ = np.histogram(img, bins=256, range=(0, 255))
    
    ## remove those value is less than 1/1000
#     for i, num in enumerate(hist):
#         if num < minimum:
#             hist[i] = 0
            
    ## otsu
    hist = 1.0*hist/np.sum(hist)
    
    val_max = -999
    thr = -1
    for t in range(1,255):
        # Non-efficient implementation
        q1 = np.sum(hist[:t])
        q2 = np.sum(hist[t:])
        m1 = np.sum(np.array([i for i in range(t)])*hist[:t])/q1
        m2 = np.sum(np.array([i for i in range(t,256)])*hist[t:])/q2
        val = q1*q2*np.power(m1-m2,2)
        if val_max < val:
            val_max = val
            thr = t
            
    print(thr)
    
    _, threshed = cv2.threshold(gray,thr,255,cv2.THRESH_BINARY_INV)
    
    return threshed

In [ ]:
# def removeBorder_2(img, height, width):

## Other Functions

In [1]:
def get_ten_mean_pics():
    images = os.listdir()
    images.sort()
    
    for i in range(0,len(images)-10):
        ten_mean = np.zeros((480,640,3), dtype=np.float64)
        for j in range(0,10):
            image = cv2.imread(images[i+j])
            image = image.astype('float64') 
            ten_mean += image/10
            
        ten_mean = np.array(np.round(ten_mean), dtype=np.uint8)
        cv2.imwrite("/home/mj/Desktop/ARCS/img/X-Ray/Ten_mean/10_mean_{}.bmp".format(i), ten_mean)